In [1]:
# 처음 실행 시
# !wget https://github.com/SwinTransformer/storage/releases/download/v1.0.2/cascade_mask_rcnn_swin_base_patch4_window7.pth
# apex 추가로 설치해야함!

In [2]:
!pip install timm

     |████████████████████████████████| 346 kB 1.9 MB/s eta 0:00:01


In [27]:
import apex

In [32]:
apex.__version__

AttributeError: module 'apex' has no attribute '__version__'

In [2]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [3]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/swin/cascade_mask_rcnn_swin_small_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py')

PREFIX = '/opt/ml/input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train_all.json'
# cfg.data.train.pipeline[2]['img_scale'] = (1333, 800)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
# cfg.data.val.pipeline[1]['img_scale'] = (1333, 800)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/swin2'
cfg.load_from = 'cascade_mask_rcnn_swin_base_patch4_window7.pth'

cfg.model.roi_head.bbox_head[0].num_classes = 11
cfg.model.roi_head.bbox_head[1].num_classes = 11
cfg.model.roi_head.bbox_head[2].num_classes = 11
cfg.model.roi_head.mask_head.num_classes = 11
# cfg.model.roi_head.bbox_head[0].num_classes=11
# cfg.model.roi_head.bbox_head[1].num_classes=11
# cfg.model.roi_head.bbox_head[2].num_classes=11
# cfg.model.roi_head.mask_head.num_classes =11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

cfg.log_config = dict(  # config to register logger hook
    interval=40,  # Interval to print the log
    hooks=[
        dict(type='TensorboardLoggerHook'),  # The Tensorboard logger is also supported
        dict(type='TextLoggerHook')
    ])
# cfg.runner = dict(type='EpochBasedRunnerAmp', max_epochs=36)
cfg.runner = dict(type='EpochBasedRunner', max_epochs=36)

# cfg.fp16 = None
# cfg.optimizer_config = dict(
#     type="DistOptimizerHook",
#     update_interval=1,
#     grad_clip=None,
#     coalesce=True,
#     bucket_size_mb=-1,
#     use_fp16=True,
# )

cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=2000,
    warmup_ratio=0.001,
    step=[27, 33])

cfg.resume_from = '/opt/ml/Swin-Transformer-Object-Detection/work_dirs/swin2/latest.pth'
cfg.checkpoint_config = dict(max_keep_ckpts=2, interval=1)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='CascadeRCNN',
    pretrained=None,
    backbone=dict(
        type='SwinTransformer',
        embed_dim=96,
        depths=[2, 2, 18, 2],
        num_heads=[3, 6, 12, 24],
        window_size=7,
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.2,
        ape=False,
        patch_norm=True,
        out_indices=(0, 1, 2, 3),
        use_checkpoint=False),
    neck=dict(
        type='FPN',
        in_channels=[96, 192, 384, 768],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_

In [3]:
model = build_detector(cfg.model)

In [4]:
# model

In [5]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=4.40s)
creating index...
index created!


In [6]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-19 19:08:05,486 - mmdet - INFO - load checkpoint from /opt/ml/Swin-Transformer-Object-Detection/work_dirs/swin2/latest.pth
2021-05-19 19:08:05,487 - mmdet - INFO - Use load_from_local loader


Done (t=0.79s)
creating index...
index created!


2021-05-19 19:08:06,640 - mmdet - INFO - resumed epoch 13, iter 21268
2021-05-19 19:08:06,643 - mmdet - INFO - Start running, host: root@cca5f118b8f3, work_dir: /opt/ml/Swin-Transformer-Object-Detection/work_dirs/swin2
2021-05-19 19:08:06,644 - mmdet - INFO - workflow: [('train', 1)], max: 36 epochs
2021-05-19 19:08:41,026 - mmdet - INFO - Epoch [14][40/1636]	lr: 1.000e-04, eta: 8:55:51, time: 0.855, data_time: 0.082, memory: 14773, loss_rpn_cls: 0.0481, loss_rpn_bbox: 0.0336, s0.loss_cls: 0.2433, s0.acc: 92.2949, s0.loss_bbox: 0.2532, s0.loss_mask: 0.2243, s1.loss_cls: 0.1178, s1.acc: 92.5420, s1.loss_bbox: 0.1015, s1.loss_mask: 0.1039, s2.loss_cls: 0.0554, s2.acc: 92.7450, s2.loss_bbox: 0.0361, s2.loss_mask: 0.0465, loss: 1.2637, grad_norm: 7.7036
2021-05-19 19:09:10,986 - mmdet - INFO - Epoch [14][80/1636]	lr: 1.000e-04, eta: 8:22:01, time: 0.749, data_time: 0.018, memory: 14773, loss_rpn_cls: 0.0394, loss_rpn_bbox: 0.0273, s0.loss_cls: 0.2141, s0.acc: 92.8491, s0.loss_bbox: 0.2308,

KeyboardInterrupt: 

In [ ]:
pwd